In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import ensemble
from sklearn.metrics import accuracy_score

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [ ]:
train['HF1'] = train['Horizontal_Distance_To_Hydrology']+train['Horizontal_Distance_To_Fire_Points']
train['HF2'] = abs(train['Horizontal_Distance_To_Hydrology']-train['Horizontal_Distance_To_Fire_Points'])
train['HR1'] = abs(train['Horizontal_Distance_To_Hydrology']+train['Horizontal_Distance_To_Roadways'])
train['HR2'] = abs(train['Horizontal_Distance_To_Hydrology']-train['Horizontal_Distance_To_Roadways'])
train['FR1'] = abs(train['Horizontal_Distance_To_Fire_Points']+train['Horizontal_Distance_To_Roadways'])
train['FR2'] = abs(train['Horizontal_Distance_To_Fire_Points']-train['Horizontal_Distance_To_Roadways'])
train['neg_ele_vert'] = train.Elevation-train.Vertical_Distance_To_Hydrology
train['ele_vert'] = train['Elevation']+train['Vertical_Distance_To_Hydrology']
#Amenities
train['Mean_Amenities']=(train.Horizontal_Distance_To_Fire_Points + train.Horizontal_Distance_To_Hydrology + train.Horizontal_Distance_To_Roadways) / 3 
#Mean Distance to Fire and Water 
train['Mean_Fire_Hyd']=(train.Horizontal_Distance_To_Fire_Points + train.Horizontal_Distance_To_Hydrology) / 2 
#Slope calc
train['slope2'] = np.sqrt(train['Horizontal_Distance_To_Hydrology']**2+train['Vertical_Distance_To_Hydrology']**2)
#Mean Hillside
train['mean_hillshade'] =  (train['Hillshade_9am']  + train['Hillshade_Noon'] + train['Hillshade_3pm'] ) / 3
#Absolute value
train["Vertical_Distance_To_Hydrology"] = abs(train['Vertical_Distance_To_Hydrology'])
train['Neg_EHyd'] = train.Elevation-train.Horizontal_Distance_To_Hydrology*0.2

In [ ]:
#Same for test
test['HF1'] = test['Horizontal_Distance_To_Hydrology']+test['Horizontal_Distance_To_Fire_Points']
test['HF2'] = abs(test['Horizontal_Distance_To_Hydrology']-test['Horizontal_Distance_To_Fire_Points'])
test['HR1'] = abs(test['Horizontal_Distance_To_Hydrology']+test['Horizontal_Distance_To_Roadways'])
test['HR2'] = abs(test['Horizontal_Distance_To_Hydrology']-test['Horizontal_Distance_To_Roadways'])
test['FR1'] = abs(test['Horizontal_Distance_To_Fire_Points']+test['Horizontal_Distance_To_Roadways'])
test['FR2'] = abs(test['Horizontal_Distance_To_Fire_Points']-test['Horizontal_Distance_To_Roadways'])
test['neg_ele_vert'] = test.Elevation-test.Vertical_Distance_To_Hydrology
test['ele_vert'] = test['Elevation']+test['Vertical_Distance_To_Hydrology']
#Amenities
test['Mean_Amenities']=(test.Horizontal_Distance_To_Fire_Points + test.Horizontal_Distance_To_Hydrology + test.Horizontal_Distance_To_Roadways) / 3 
#Mean Distance to Fire and Water 
test['Mean_Fire_Hyd']=(test.Horizontal_Distance_To_Fire_Points + test.Horizontal_Distance_To_Hydrology) / 2 
#Slope calc
test['slope2'] = np.sqrt(test['Horizontal_Distance_To_Hydrology']**2+test['Vertical_Distance_To_Hydrology']**2)
#Mean Hillside
test['mean_hillshade'] =  (test['Hillshade_9am']  + test['Hillshade_Noon'] + test['Hillshade_3pm'] ) / 3
#Absolute value
test["Vertical_Distance_To_Hydrology"] = abs(test['Vertical_Distance_To_Hydrology'])
test['Neg_EHyd'] = test.Elevation-test.Horizontal_Distance_To_Hydrology*0.2

In [ ]:
train.drop(['Id'], inplace = True, axis = 1 )
train.drop(['Soil_Type15' , "Soil_Type7"], inplace = True, axis = 1 )
test.drop(['Soil_Type15' , "Soil_Type7"], inplace = True, axis = 1 )

In [ ]:
cols=[i for i in train.columns if i not in ['Id', 'Cover_Type' ]]

In [ ]:
X=train[cols]
Y=train.Cover_Type
clf=RandomForestClassifier(n_estimators=500,verbose = 1,n_jobs=-1,oob_score=True,random_state=0,warm_start=True)
clf.fit(X, Y)

In [ ]:
feature_importances = pd.DataFrame(clf.feature_importances_,
                                   index = X.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)

In [ ]:
feature_importances

In [ ]:
scaler = StandardScaler()


In [ ]:
feature = [col for col in train.columns if col not in ['Cover_Type']]
X = train[feature]
scaler.fit(X)
X = scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.3)

In [ ]:
etc = ensemble.ExtraTreesClassifier(n_estimators=1000)  
etc.fit(X_train, y_train)
y_pred = etc.predict(X_test)

In [ ]:
print(accuracy_score(y_test,y_pred))

In [ ]:
feature = [col for col in train.columns if col not in ['Cover_Type']]
X = test[feature]
X = scaler.transform(X)

In [ ]:
y_pred = etc.predict(X)

In [ ]:
sub = {'Id':test["Id"].values,'Cover_Type':y_pred}

In [ ]:
sub = pd.DataFrame(data=sub)

In [ ]:
sub

In [ ]:
sub.to_csv("ExtraTreesClassifier.csv", index=False)